# Project 1
August Noer Steinset

## Setting up a local database

Setting up cassandra and spark

In [1]:
import os
from pyspark.sql import SparkSession

os.environ["PYSPARK_PYTHON"] = "python" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "python"

# Connecting to Cassandra
from cassandra.cluster import Cluster
cluster = Cluster(['localhost'], port=9042)
session = cluster.connect()

spark = SparkSession.builder.appName('SparkCassandraApp').\
    config('spark.jars.packages', 'com.datastax.spark:spark-cassandra-connector_2.12:3.4.1').\
    config('spark.cassandra.connection.host', 'localhost').\
    config('spark.sql.extensions', 'com.datastax.spark.connector.CassandraSparkExtensions').\
    config('spark.sql.catalog.mycatalog', 'com.datastax.spark.connector.datasource.CassandraCatalog').\
    config('spark.cassandra.connection.port', '9042').getOrCreate()

Creating some data with cassandra and showing it with spark

In [2]:
# Deleting the keyspace and the table
session.execute("DROP TABLE IF EXISTS project1.table1")
session.execute("DROP TABLE IF EXISTS project1.table2")
session.execute("DROP TABLE IF EXISTS project1.table3")
session.execute("DROP KEYSPACE IF EXISTS project1")

# Creating a keyspace
session.execute("CREATE KEYSPACE IF NOT EXISTS project1 WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 1}")
session.set_keyspace('project1')

# Creating a table
session.execute("CREATE TABLE IF NOT EXISTS project1.table1 (id int PRIMARY KEY, name text, age int, city text, job text)")

# Inserting data into the table
session.execute("INSERT INTO project1.table1 (id, name, age, city, job) VALUES (1, 'John', 25, 'London', 'Engineer')")
session.execute("INSERT INTO project1.table1 (id, name, age, city, job) VALUES (2, 'Anna', 22, 'Paris', 'Teacher')")
session.execute("INSERT INTO project1.table1 (id, name, age, city, job) VALUES (3, 'Peter', 29, 'Berlin', 'Doctor')")
session.execute("INSERT INTO project1.table1 (id, name, age, city, job) VALUES (4, 'Linda', 33, 'New York', 'Lawyer')")
session.execute("INSERT INTO project1.table1 (id, name, age, city, job) VALUES (5, 'John', 42, 'Paris', 'Engineer')")
session.execute("INSERT INTO project1.table1 (id, name, age, city, job) VALUES (6, 'Anna', 28, 'London', 'Teacher')")

# Creating a dataframe
df = spark.read.format("org.apache.spark.sql.cassandra").options(table="table1", keyspace="project1").load()
df.show()

# Creating a view
df.createOrReplaceTempView("table1")

# Querying the view
df2 = spark.sql("SELECT * FROM table1 WHERE age > 30")
df2.show()

# Deleting the keyspace and the table
session.execute("DROP TABLE IF EXISTS project1.table1")
session.execute("DROP KEYSPACE IF EXISTS project1")

# See if the table is still there
df = spark.read.format("org.apache.spark.sql.cassandra").options(table="table1", keyspace="project1").load()
try:
    df.show()
except:
    print("The table is not there anymore.")



+---+---+--------+--------+-----+
| id|age|    city|     job| name|
+---+---+--------+--------+-----+
|  4| 33|New York|  Lawyer|Linda|
|  6| 28|  London| Teacher| Anna|
|  2| 22|   Paris| Teacher| Anna|
|  5| 42|   Paris|Engineer| John|
|  1| 25|  London|Engineer| John|
|  3| 29|  Berlin|  Doctor|Peter|
+---+---+--------+--------+-----+

+---+---+--------+--------+-----+
| id|age|    city|     job| name|
+---+---+--------+--------+-----+
|  4| 33|New York|  Lawyer|Linda|
|  5| 42|   Paris|Engineer| John|
+---+---+--------+--------+-----+

The table is not there anymore.


## Data download

In [3]:
# Read from file secrets.txt and store the client id and client secret
import requests
with open('secrets.txt', 'r') as f:
	client_id = f.readline().strip()
	client_secret = f.readline().strip()

config = {
	'client_id': client_id,
	'client_secret': client_secret,
	'token_url': 'https://id.barentswatch.no/connect/token',
	'api_base_url': 'https://www.barentswatch.no/bwapi'
}

def get_token():

  if not config['client_id']:
    raise ValueError('client_id must be set in credentials.py')

  if not config['client_secret']:
    raise ValueError('client_secret must be set in credentials.py')

  req = requests.post(config['token_url'],
    data={
        'grant_type': 'client_credentials',
        'client_id': config['client_id'],
        'client_secret': config['client_secret'],
        'scope': 'api'
    },
    headers={'content-type': 'application/x-www-form-urlencoded'})

  req.raise_for_status()
  print('Token request successful')
  return req.json()



In [4]:
def get_week_summary(token, year, week):
  url = f"{config['api_base_url']}/v1/geodata/fishhealth/locality/{year}/{week}"
  headers ={
    'authorization': 'Bearer ' + token['access_token'],
    'content-type': 'application/json',
  }

  response = requests.get(url, headers=headers)
  response.raise_for_status()
  return response.json()

def get_localities(token):
  url = f"{config['api_base_url']}/v1/geodata/fishhealth/localities"
  headers ={
    'authorization': 'Bearer ' + token['access_token'],
    'content-type': 'application/json',
  }
  response = requests.get(url, headers=headers)
  response.raise_for_status()
  return response.json()


token = get_token()
locality = get_localities(token)
weeksummary= get_week_summary(token,'2017','45')

Token request successful


In [5]:

import numpy as np
#set seed
np.random.seed(12345678)
# pick a locality number from the list of localities
random_location = np.random.choice(locality)
random_location['localityNo']


12253

In [6]:
# get a year worth of data
def get_year_summary(token, year):
    year_list = []
    for i in range(52):
        weeksummary = get_week_summary(token, year, str(i+1))
        year_list.append(weeksummary)
    return year_list

token = get_token()
yearsummary = get_year_summary(token, '2020')


Token request successful


In [7]:
# Select one locality and download lice count data (localityWeek) for the same year from: /v1/geodata/fishhealth/locality/{localityNo}/{week}

def get_locality_week(token, localityNo, year, week):
    url = f"{config['api_base_url']}/v1/geodata/fishhealth/locality/{localityNo}/{year}/{week}"
    headers ={
        'authorization': 'Bearer ' + token['access_token'],
        'content-type': 'application/json',
    }
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()


def get_year_locality(token, localityNo, year):
    year_list = []
    for i in range(52):
        weeksummary = get_locality_week(token, localityNo, year, str(i+1))
        year_list.append(weeksummary)
    return year_list

token = get_token()
localityyear = get_year_locality(token, random_location['localityNo'], '2020')

Token request successful


In [8]:
session.execute("CREATE KEYSPACE IF NOT EXISTS project1 WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 1}")
session.set_keyspace('project1')

# Creating a table
session.execute("CREATE TABLE IF NOT EXISTS project1.table1 (aquaCultureRegistryVersion int, localityNo int PRIMARY KEY, name text, municipalityNo int, municipality text)")
session.execute("CREATE TABLE IF NOT EXISTS project1.table2 (year int, week int PRIMARY KEY, localities blob)")
session.execute("CREATE TABLE IF NOT EXISTS project1.table3 (year int,	week int PRIMARY KEY, hasReportedLice boolean, avgAdultFemaleLice float, avgMobileLice float, avgStationaryLice float)")


In [9]:
# convert locality(list of dictionaries) to dataframe
import pandas as pd
locality_df = pd.DataFrame(locality)
locality_df.columns = locality_df.columns.str.lower()

spark.createDataFrame(locality_df).write.format("org.apache.spark.sql.cassandra")\
.options(table="table1", keyspace="project1").mode("append").save()

c:\Users\augus\miniconda3\envs\IND320\Lib\site-packages\pyspark\sql\pandas\conversion.py:485: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if should_localize and is_datetime64tz_dtype(s.dtype) and s.dt.tz is not None:


In [10]:
df = spark.read.format("org.apache.spark.sql.cassandra").options(table="table1", keyspace="project1").load()
df.show()

+----------+--------------------------+-------------+--------------+------------+
|localityno|aquacultureregistryversion| municipality|municipalityno|        name|
+----------+--------------------------+-------------+--------------+------------+
|     32077|                       429|         AURE|          1576|       Aukan|
|     13185|                      1235|        DØNNA|          1827|   Bjørnsøya|
|     45195|                      1235|    FLATANGER|          5049|   Lamholmen|
|     45047|                      1235|        RAUMA|          1539|   Geilbukta|
|     10110|                      1235|    STAVANGER|          1103|    Djupevik|
|     14202|                       139|    BREMANGER|          1438|Tennøysundet|
|     37597|                      1235|  FLEKKEFJORD|          4207|   Guleodden|
|     22435|                      1235|       HADSEL|          1866|Stretarneset|
|     45114|                      1235|   HUSTADVIKA|          1579|Ramsvikneset|
|     32537|    

In [12]:
df_year = pd.DataFrame(yearsummary)
df_year.columns = df_year.columns.str.lower()


spark.createDataFrame(df_year).write.format("org.apache.spark.sql.cassandra")\
.options(table="table2", keyspace="project1").mode("append").save()

df_year.head()

c:\Users\augus\miniconda3\envs\IND320\Lib\site-packages\pyspark\sql\pandas\conversion.py:485: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if should_localize and is_datetime64tz_dtype(s.dtype) and s.dt.tz is not None:


Py4JJavaError: An error occurred while calling o139.save.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 5 in stage 11.0 failed 1 times, most recent failure: Lost task 5.0 in stage 11.0 (TID 106) (10.20.1.142 executor driver): com.datastax.spark.connector.types.TypeConversionException: Cannot convert object WrappedArray(Map(hasPd -> null, name -> null, hasIla -> null, inFilteredSelection -> null, hasReportedLice -> null, isOnLand -> null, hasSalmonoids -> null, hasSubstanceTreatments -> null, isSlaughterHoldingCage -> null, isFallow -> null, localityWeekId -> 1075043, lon -> null, municipalityNo -> null, hasMechanicalRemoval -> null, avgAdultFemaleLice -> null, municipality -> null, localityNo -> 14746, hasCleanerfishDeployed -> null, lat -> null), Map(hasPd -> null, name -> null, hasIla -> null, inFilteredSelection -> null, hasReportedLice -> null, isOnLand -> null, hasSalmonoids -> null, hasSubstanceTreatments -> null, isSlaughterHoldingCage -> null, isFall

In [ ]:
df = spark.read.format("org.apache.spark.sql.cassandra").options(table="table2", keyspace="project1").load()
df.show()

In [ ]:
# convert localityyear(list of dictionaries) to dataframe
import pandas as pd
df = pd.DataFrame(localityyear)
df_localityyear = pd.json_normalize(df['liceCountPreviousWeek'])
df_localityyear.columns = df_localityyear.columns.str.lower()

df_localityyear.head()

df_localityyear = df_localityyear.astype({'avgadultfemalelice': 'float64', 'avgmobilelice': 'float64', 'avgstationarylice': 'float64', 'hasreportedlice': 'bool', 'week': 'int64', 'year': 'int64'})

spark.createDataFrame(df_localityyear).write.format("org.apache.spark.sql.cassandra")\
.options(table="table3", keyspace="project1").mode("append").save()


In [ ]:
df = spark.read.format("org.apache.spark.sql.cassandra").options(table="table3", keyspace="project1").load()
df.show()

# plots

In [ ]:
import json
import ast
from pyspark.sql.functions import explode, col
# plot proportions of localities that have reported Pancreas Disease (PD/Pd) during the chosen year

sdf = spark.read.format("org.apache.spark.sql.cassandra").options(table="table2", keyspace="project1").load()

# Explode the localities array
sdf_exploded = sdf.select("week", "year", explode("localities").alias("locality"))

# Count the occurrences of each hasPd value
result = sdf_exploded.groupBy("locality.hasPd").count().show()

# Output the result
result.show()

# Deleting the keyspace and the table
session.execute("DROP TABLE IF EXISTS project1.table1")
session.execute("DROP TABLE IF EXISTS project1.table2")
session.execute("DROP TABLE IF EXISTS project1.table3")
session.execute("DROP KEYSPACE IF EXISTS project1")

# Closing the connection
session.shutdown()
cluster.shutdown()
spark.stop()